In [1]:
import pandas as pd

# This 'latin-1' part fixes the error you just saw!
df = pd.read_csv('DataCoSupplyChainDataset.csv', encoding='latin-1')

# Show the first 3 rows to prove it worked
pd.set_option('display.max_columns', None)
print(df.head(3))

       Type  Days for shipping (real)  Days for shipment (scheduled)  \
0     DEBIT                         3                              4   
1  TRANSFER                         5                              4   
2      CASH                         4                              4   

   Benefit per order  Sales per customer   Delivery Status  \
0          91.250000          314.640015  Advance shipping   
1        -249.089996          311.359985     Late delivery   
2        -247.779999          309.720001  Shipping on time   

   Late_delivery_risk  Category Id   Category Name Customer City  \
0                   0           73  Sporting Goods        Caguas   
1                   1           73  Sporting Goods        Caguas   
2                   0           73  Sporting Goods      San Jose   

  Customer Country Customer Email Customer Fname  Customer Id Customer Lname  \
0      Puerto Rico      XXXXXXXXX          Cally        20755       Holloway   
1      Puerto Rico      XXXXX

In [2]:
df.columns = [col.lower().replace(' ', '_').replace('(', '').replace(')', '') for col in df.columns]

df['order_date_dateorders'] = pd.to_datetime(df['order_date_dateorders'])
df['shipping_date_dateorders'] = pd.to_datetime(df['shipping_date_dateorders'])

cols_to_drop = ['customer_email', 'customer_password', 'product_image', 'product_description']
df = df.drop(columns=cols_to_drop)

print("Step 1 Complete! New Column Names:")
print(df.columns[0:5]) 

Step 1 Complete! New Column Names:
Index(['type', 'days_for_shipping_real', 'days_for_shipment_scheduled',
       'benefit_per_order', 'sales_per_customer'],
      dtype='object')


In [3]:
df['delay_days'] = df['days_for_shipping_real'] - df['days_for_shipment_scheduled']

df['delivery_performance'] = df['delay_days'].apply(lambda x: 'Late' if x > 0 else ('On Time' if x == 0 else 'Early'))

late_rate = df['late_delivery_risk'].mean() * 100

print(f"Step 2 Complete! Overall Late Delivery Rate: {late_rate:.2f}%")
print(df[['delivery_status', 'days_for_shipping_real', 'days_for_shipment_scheduled', 'delivery_performance']].head())

Step 2 Complete! Overall Late Delivery Rate: 54.83%
    delivery_status  days_for_shipping_real  days_for_shipment_scheduled  \
0  Advance shipping                       3                            4   
1     Late delivery                       5                            4   
2  Shipping on time                       4                            4   
3  Advance shipping                       3                            4   
4  Advance shipping                       2                            4   

  delivery_performance  
0                Early  
1                 Late  
2              On Time  
3                Early  
4                Early  


In [4]:
 
print("Columns currently in df:")
for col in df.columns:
    print(col)

Columns currently in df:
type
days_for_shipping_real
days_for_shipment_scheduled
benefit_per_order
sales_per_customer
delivery_status
late_delivery_risk
category_id
category_name
customer_city
customer_country
customer_fname
customer_id
customer_lname
customer_segment
customer_state
customer_street
customer_zipcode
department_id
department_name
latitude
longitude
market
order_city
order_country
order_customer_id
order_date_dateorders
order_id
order_item_cardprod_id
order_item_discount
order_item_discount_rate
order_item_id
order_item_product_price
order_item_profit_ratio
order_item_quantity
sales
order_item_total
order_profit_per_order
order_region
order_state
order_status
order_zipcode
product_card_id
product_category_id
product_name
product_price
product_status
shipping_date_dateorders
shipping_mode
delay_days
delivery_performance


In [5]:
import numpy as np
from math import radians, cos, sin, asin, sqrt

# 1. Define the Haversine Formula  
def haversine(lon1, lat1, lon2, lat2):
    # Convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371 # Radius of earth in kilometers
    return c * r

# 2. Define Central Warehouse Location (Puerto Rico HQ)
warehouse_lat = 18.2208
warehouse_lon = -66.5901

# 3. Applyiing  the Formula (Using the FIXED warehouse coordinates)
#  calculating distance from Warehouse -> Customer (row['latitude'], row['longitude'])
df['delivery_distance_km'] = df.apply(lambda row: haversine(
    warehouse_lon, warehouse_lat,
    row['longitude'], row['latitude']
), axis=1)

# Sanity Check
print("Step 3 Fixed! Distance calculated from HQ.")
print(df[['order_city', 'delivery_distance_km']].head())

Step 3 Fixed! Distance calculated from HQ.
   order_city  delivery_distance_km
0      Bekasi             58.506413
1     Bikaner             58.764545
2     Bikaner           5749.199740
3  Townsville           5391.937991
4  Townsville             58.522394


In [6]:
%pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.2 -> 26.0.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
 from sklearn.preprocessing import LabelEncoder

 
le = LabelEncoder()

  
df['shipping_mode_encoded'] = le.fit_transform(df['shipping_mode'])
 
df['order_region_encoded'] = le.fit_transform(df['order_region'])

 
print("Step 4 Complete! Data is ready for ML.")
print(df[['shipping_mode', 'shipping_mode_encoded']].head())

Step 4 Complete! Data is ready for ML.
    shipping_mode  shipping_mode_encoded
0  Standard Class                      3
1  Standard Class                      3
2  Standard Class                      3
3  Standard Class                      3
4  Standard Class                      3


In [10]:
 
 
dim_customers = df[['customer_id', 'customer_fname', 'customer_lname', 
                    'customer_city', 'customer_country', 'customer_segment']].drop_duplicates()

 
dim_products = df[['product_card_id', 'product_name', 'category_name', 
                   'product_price', 'product_status']].drop_duplicates()
 
 
fact_supply_chain = df[['order_id', 'order_date_dateorders', 'customer_id', 'product_card_id', 
                        'type', 'shipping_mode_encoded', 'delivery_distance_km', 
                        'late_delivery_risk', 'sales', 'order_region']]

 
dim_customers.to_csv('Dim_Customers.csv', index=False)
dim_products.to_csv('Dim_Products.csv', index=False)
fact_supply_chain.to_csv('Fact_Supply_Chain.csv', index=False)

print( "Data split into 3 tables. Ready for SQL import.")

Data split into 3 tables. Ready for SQL import.


In [12]:
from sqlalchemy import create_engine
import urllib

# 1. Define Connection (Change 'YourStrong!Pass123' to your actual password)
# Note: We use 'mssql+pyodbc' to talk to SQL Server
params = urllib.parse.quote_plus(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=localhost;"
    "DATABASE=SupplyChainDB;"
    "UID=sa;"
    "PWD=StrongPass@123"
)

# 2. Connect
engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

# 3. Push the tables!
# 'if_exists="replace"' means it will auto-create the table for you!
print("Uploading Customers...")
dim_customers.to_sql('Dim_Customers', engine, if_exists='replace', index=False)

print("Uploading Products...")
dim_products.to_sql('Dim_Products', engine, if_exists='replace', index=False)

print("Uploading Fact Table (This takes a moment)...")
fact_supply_chain.to_sql('Fact_Supply_Chain', engine, if_exists='replace', index=False)

print("Done! Data is now in SQL Server.")

Uploading Customers...
Uploading Products...
Uploading Fact Table (This takes a moment)...
Done! Data is now in SQL Server.
